In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-04-19 16:32:45--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2024-04-19 16:32:46 (176 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [14]:
# merge the two datasets
df = pd.merge(df_ratings, df_books, on='isbn')

# group by isbn and count number of ratings
df_rating_counts = df.groupby('isbn')['rating'].count().reset_index()
df_rating_counts.columns = ['isbn', 'rating_count']

# filter out books with less than 100 ratings
df = df[df['isbn'].isin(df_rating_counts.loc[df_rating_counts['rating_count'] >= 100, 'isbn'])]

# group by user and count number of ratings
df_user_counts = df.groupby('user')['rating'].count().reset_index()
df_user_counts.columns = ['user', 'rating_count']

# filter out users with less than 200 ratings
df = df[df['user'].isin(df_user_counts.loc[df_user_counts['rating_count'] >= 200, 'user'])]

# create a pivot table with isbn as index, user as column, and rating as value
df_pivot = df.pivot_table(index='isbn', columns='user', values='rating').fillna(0)

# TEST print the shape of df_pivot.values
print(df_pivot.values.shape)

# Drop columns with all NaN values
df_pivot.dropna(axis=1, how='all', inplace=True)

# convert the pivot table to a sparse matrix
sparse_matrix = csr_matrix(df_pivot.values)

# TEST print the shape of sparse_matrix
print(sparse_matrix.shape)

# create a nearest neighbors model
model = NearestNeighbors(algorithm='brute', metric='cosine')
model.fit(sparse_matrix.T)  # Fit the model with the transpose of the sparse matrix

(727, 20)
(727, 20)


NearestNeighbors(algorithm='brute', metric='cosine')

In [16]:
# function to return recommended books
def get_recommends(book_title):
    # Check if book title exists in the data
    book_exists = book_title in df_books['title'].tolist()

    if not book_exists:
        raise ValueError("Book title not found in the data.")

    # get the isbn of the book
    book_isbn = df_books.loc[df_books['title'] == book_title, 'isbn'].iloc[0]
    # TEST print book title and ISBN
    print(book_isbn)
    print(book_isbn in df_pivot.index)

    # get the index of the book in the pivot table
    book_index = df_pivot.index.tolist().index(book_isbn)

    # get the nearest neighbors
    distances, indices = model.kneighbors(sparse_matrix.T[book_index].reshape(1, -1), n_neighbors=6)

    # get the recommended books
    recommended_books = []
    for i in range(1, 6):
        if indices[0][i] < len(df_pivot.index):  # Check if the index is within the range
            recommended_book_isbn = df_pivot.index[indices[0][i]]
            recommended_book_title = df_books.loc[df_books['isbn'] == recommended_book_isbn, 'title'].iloc[0]
            recommended_books.append([recommended_book_title, distances[0][i]])

    return [book_title, recommended_books]

In [17]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

0446672211
True


IndexError: row index (434) out of range